In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('/home/zagreus/Programming/PPMI_Research_on_Parkinsons/src/mahbub/processed_ppmi_master_dataset.csv')

In [33]:
df

,Patient ID,Visit Date,EVENT_ID,NP3SPCH,NP3FACXP,NP3RIGN,NP3RIGRU,NP3RIGLU,NP3RIGRL,NP3RIGLL,...,correlation,dissimilarity,energy,gabor_energy,gabor_entropy,homogeneity,lbp_energy,lbp_entropy,VisitNumber,Visit Class
0,3010,2011-07-13,BL,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,0.831520,0.816963,0.989929,9.525620,379.868298,0.995363,0.990573,0.054468,1,0 months
1,3010,2012-06-26,V04,1.0,2.0,1.0,0.0,2.0,0.0,0.0,...,0.852982,1.562270,0.977459,17.621092,781.401212,0.991459,0.983486,0.089167,2,5-17 months
2,3010,2013-07-17,V06,2.0,2.0,1.0,0.0,2.0,0.0,1.0,...,0.882104,0.894156,0.985684,13.268292,519.371246,0.994178,0.988597,0.064722,3,18-30 months
3,3010,2015-06-26,V10,1.0,2.0,0.0,0.0,2.0,0.0,0.0,...,0.837949,1.390070,0.982016,16.052184,658.612867,0.992227,0.984798,0.082195,4,31-65 months
4,3012,2011-10-11,SC,0.0,2.0,0.0,1.0,0.0,0.0,0.0,...,0.899377,0.768224,0.984650,15.227890,573.424163,0.996028,0.990869,0.053686,1,0 months
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,4123,2017-03-01,V10,0.0,0.0,2.0,2.0,1.0,1.0,1.0,...,0.881892,1.713302,0.971745,20.641646,953.635778,0.989083,0.979675,0.107821,4,31-65 months
860,4126,2013-03-07,V01,2.0,2.0,2.0,2.0,1.0,0.0,0.0,...,0.788101,1.763254,0.976939,17.137963,761.007928,0.990175,0.982297,0.091230,1,0 months
861,4126,2014-04-24,V04,1.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.960928,0.521967,0.977298,17.823609,755.169460,0.996123,0.989590,0.059155,2,5-17 months
862,4126,2015-04-15,V06,0.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.946256,0.703124,0.977285,18.181537,784.740034,0.995052,0.987884,0.069876,3,18-30 months


In [34]:
# df.dtypes

In [35]:
df['Visit Date'] = pd.to_datetime(df['Visit Date'])

# Calculate the first visit date for each patient
df['first_visit_date'] = df.groupby('Patient ID')['Visit Date'].transform('min')

# Calculate the interval in months from the first visit date
df['month_interval'] = df.apply(
    lambda row: (row['Visit Date'].year - row['first_visit_date'].year) * 12 + (row['Visit Date'].month - row['first_visit_date'].month),
    axis=1
)

# Drop the first_visit_date column if it's not needed
df = df.drop(columns=['first_visit_date'])

# Reorder columns to place month_interval next to Visit Date
cols = list(df.columns)
visit_date_index = cols.index('Visit Date')

# Move month_interval to the position after Visit Date
cols.insert(visit_date_index + 1, cols.pop(cols.index('month_interval')))

# Move 'Visit Class' to the position after 'month_interval'
cols.insert(visit_date_index + 2, cols.pop(cols.index('Visit Class')))

# Move 'VisitNumber' to the position after 'Visit Class'
cols.insert(visit_date_index + 3, cols.pop(cols.index('VisitNumber')))

df = df[cols]

In [36]:
mapping = {
    '0 months': 'V1',
    '5-17 months': 'V2',
    '18-30 months': 'V3',
    '31-65 months': 'V4'
}

df['Visit'] = df['Visit Class'].map(mapping)

# Get the list of columns
cols = list(df.columns)

# Find the index of 'month_interval'
month_interval_index = cols.index('month_interval')

# Move 'Visit' to the position after 'month_interval'
cols.insert(month_interval_index + 1, cols.pop(cols.index('Visit')))

# Reorder the DataFrame
df = df[cols]

In [37]:
df

,Patient ID,Visit Date,month_interval,Visit,Visit Class,VisitNumber,EVENT_ID,NP3SPCH,NP3FACXP,NP3RIGN,...,brightness,contrast,correlation,dissimilarity,energy,gabor_energy,gabor_entropy,homogeneity,lbp_energy,lbp_entropy
0,3010,2011-07-13,0,V1,0 months,1,BL,0.0,1.0,1.0,...,0.007926,158.463741,0.831520,0.816963,0.989929,9.525620,379.868298,0.995363,0.990573,0.054468
1,3010,2012-06-26,11,V2,5-17 months,2,V04,1.0,2.0,1.0,...,0.018020,326.583983,0.852982,1.562270,0.977459,17.621092,781.401212,0.991459,0.983486,0.089167
2,3010,2013-07-17,24,V3,18-30 months,3,V06,2.0,2.0,1.0,...,0.012063,178.744707,0.882104,0.894156,0.985684,13.268292,519.371246,0.994178,0.988597,0.064722
3,3010,2015-06-26,47,V4,31-65 months,4,V10,1.0,2.0,0.0,...,0.014601,296.968707,0.837949,1.390070,0.982016,16.052184,658.612867,0.992227,0.984798,0.082195
4,3012,2011-10-11,0,V1,0 months,1,SC,0.0,2.0,0.0,...,0.013327,167.855630,0.899377,0.768224,0.984650,15.227890,573.424163,0.996028,0.990869,0.053686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,4123,2017-03-01,49,V4,31-65 months,4,V10,0.0,0.0,2.0,...,0.023252,333.979968,0.881892,1.713302,0.971745,20.641646,953.635778,0.989083,0.979675,0.107821
860,4126,2013-03-07,0,V1,0 months,1,V01,2.0,2.0,2.0,...,0.017197,400.462547,0.788101,1.763254,0.976939,17.137963,761.007928,0.990175,0.982297,0.091230
861,4126,2014-04-24,13,V2,5-17 months,2,V04,1.0,2.0,1.0,...,0.021192,102.895627,0.960928,0.521967,0.977298,17.823609,755.169460,0.996123,0.989590,0.059155
862,4126,2015-04-15,25,V3,18-30 months,3,V06,0.0,2.0,1.0,...,0.021157,142.389797,0.946256,0.703124,0.977285,18.181537,784.740034,0.995052,0.987884,0.069876


In [38]:
df['Visit'].value_counts()

Visit
V1    216
V2    216
V3    216
V4    216
Name: count, dtype: int64

In [39]:
df.dtypes

Patient ID                 int64
Visit Date        datetime64[ns]
month_interval             int64
Visit                     object
Visit Class               object
                       ...      
gabor_energy             float64
gabor_entropy            float64
homogeneity              float64
lbp_energy               float64
lbp_entropy              float64
Length: 94, dtype: object

In [40]:
df.to_csv('CreatingDataset/processed_ppmi_master_dataset_FINAL_37_39.csv', index=False)